In [1]:
import re
import json
import pysrt
import pandas as pd

def parse_ass_file(file_path):
    events = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    event_section = False
    for line in lines:
        if line.strip().startswith('[Events]'):
            event_section = True
            continue
        if event_section:
            if line.strip().startswith('Dialogue:'):
                # Structure Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
                events.append(line.strip().replace("\\N", "\n").split(",", maxsplit=9))
                    
    return events

def extract_subtitles(file_path):
    # Load the SRT file
    subtitles = pysrt.open(file_path)

    # Extract the text from each subtitle
    subtitle_texts = [sub.text for sub in subtitles]

    return subtitle_texts

# Use the function
# subtitles = extract_subtitles("/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E008 (R2DVD 960x728 x264 AC3).en.srt")
# content = "\n".join(subtitles)

In [2]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [3]:
llm_url = "http://localhost:1234/v1"
api_key = "lm-studio"
model_name = "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF"
model_name = "bartowski/aya-23-8B-GGUF"
model_name = "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF"
model_name = "RichardErkhov/SanjiWatsuki_-_Silicon-Maid-7B-gguf" # Good for manga, not followed instruction well
model_name = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

In [106]:
prompt_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
# Instruction
## Summarize events in this subtitle
In clear and concise language, summarize chunk of sentences will take about the interactive events among the characters.

## Generate 
Generate summary events, characters info, why it happened, where it happened and how it happened. If possible, mention in detail about jutsu and movement.

## Write output
Here is structure format:
- characters:
  - name
  - role
  - background
  - current status
- events
  - description
  - characters_involved
- reason_for_events
  - description
  - event
- location
- jutsu
  - name
  - character
  - description
Write comprehensive output in JSON format

# Guideline
- Both contents are in English
- Be informative, concise
- Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

### Input
{content}
"""
test_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
### Instruction
#### Summarize events in this subtitle
In clear and concise language, summarize chunk of sentences will take about the interactive events among the characters.

#### Generate 
Generate summary events, characters info, why it happened, where it happened and how it happened. If possible, mention in detail about jutsu and movement.

### Write output
Write comprehensive output following by structure below
- characters:
  - name
  - role
  - background
  - current status
- events
  - description
  - characters_involved
- reason_for_events
  - description
  - event
- location
- jutsu
  - name
  - character
  - description

### Guideline
- Both contents are in English
- Be informative, concise
- Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

### Input
{prev}
{content}
"""
silicon_maid_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
### Instruction
#### Summarize events in this subtitle
In clear and concise language, summarize chunk of sentences will take about the interactive events among the characters.

#### Generate 
Generate summary events, characters info, why it happened, where it happened and how it happened. If possible, mention in detail about jutsu and movement.

### Write output
Write comprehensive output following by structure below
- characters:
  - name
  - role
  - background
  - current status
- events
  - description
  - characters_involved
- reason_for_events
  - description
  - event
- location
- jutsu
  - name
  - character
  - description

### Guideline
- Both contents are in English
- Be informative, concise
- Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

### Input
{content}

### Response
"""

chain_of_density_template = """
I will provide you with piece of content (e.g. articles, papers, documentation, etc.)
You will generate increasingly concise, entity-dense summaries of the content.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.

Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:

Relevant: to the main story.
Specific: descriptive yet concise (5 words or fewer).
Novel: not in the previous summary.
Faithful: present in the content piece.
Anywhere: located anywhere in the Article.

Guidelines:

The first summary should be long (5 sentences, - 80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach - 80 words.
Make every word count: re-write the previous summary to improve flow and make space for additional entities.
Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
Missing entities can appear anywhere in the new summary.
Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary"

### Input
{content}
"""

def get_chain(llm_url, model_name, template, temperature: float = 0.5, max_tokens: int = -1):
    client = ChatOpenAI(
            temperature=temperature,
            base_url=llm_url,
            model_name=model_name,
            api_key=api_key,
            max_tokens=max_tokens,
        )
    map_prompt = PromptTemplate.from_template(template)
    return LLMChain(llm=client, prompt=map_prompt)

In [19]:
# llm_chain = get_chain(llm_url, model_name, prompt_template)
# llm_chain = get_chain(llm_url, model_name, silicon_maid_template)
llm_chain = get_chain(llm_url, model_name, chain_of_density_template, temperature=0.1)

In [98]:
# previous_episodes = 1
prev, content = "" , ""
current_episode = 132
# for episode in range(current_episode - previous_episodes, current_episode):
#     ass_file_path = "/Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - {episode} [720p].ass".format(episode=episode)
#     dialogues = parse_ass_file(ass_file_path)
#     df_dialogue = pd.DataFrame(dialogues, columns=["Layer", "Start", "End", "Style", "Name", "MarginL", "MarginR", "MarginV", "Effect","Text"])
#     prev += "\n".join(df_dialogue["Text"].tolist())

ass_file_path = "/Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - {episode} [720p].ass".format(episode=current_episode)
dialogues = parse_ass_file(ass_file_path)
df_dialogue = pd.DataFrame(dialogues, columns=["Layer", "Start", "End", "Style", "Name", "MarginL", "MarginR", "MarginV", "Effect","Text"])
content += "\n".join(df_dialogue["Text"].tolist())
# # prompt_render = test_template.format(prev=prev, content=content)
# # print(llm_chain.invoke(content)["text"])

In [6]:
subtitles = extract_subtitles("/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E001 (R2DVD 960x728 x264 AC3).en.srt")
content = "\n".join(subtitles)

## Chain of density
Does not work well, skip it

In [60]:
chain_of_density_template = """
### Instruction
I will provide you with piece of content (e.g. articles, papers, documentation, etc.)
You will generate increasingly concise, entity-dense summaries of the content.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.

Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:

Relevant: to the main story.
Specific: descriptive yet concise (5 words or fewer).
Novel: not in the previous summary.
Faithful: present in the content piece.
Anywhere: located anywhere in the Article.

### Guidelines:

The first summary should be long (5 sentences) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach - 80 words.
Make every word count: re-write the previous summary to improve flow and make space for additional entities.
Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
Missing entities can appear anywhere in the new summary.
Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary.

Answer in YAML. The YAML should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary". YAML output only, no further explanation needed.

# Example Output
- Missing_Entities: "Naruto; Sasuke; Sakura"
  Denser_Summary: Summary
"""

In [61]:
import json
import yaml


def extract_entities(summary):
    try:
        yaml_output = summary[summary.find("-") :]
        outputs = yaml.safe_load(yaml_output)
        return [output["Missing_Entities"] for output in outputs]
    except Exception as e:
        print(summary, e)
        return []


def summarize_cod(client, article, summary_steps=3, temperature=0.1):
    summary_chain = []
    output = client.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": silicon_maid_template.format(content=article)
            },
        ],
        temperature=temperature
    )
    prev_summary = None
    summary_chain.append(output.choices[0].message.content)

    for i in range(summary_steps):
        missing_entities = extract_entities(prev_summary) if prev_summary else []
        missing_entities = ";".join(missing_entities)
        output = client.create(
            model=model_name,
            temperature=temperature,
            messages=[
                {
                    "role": "system",
                    "content": chain_of_density_template,
                },
                {   
                    "role": "user", 
                    "content": f"Here is the Article: {article}"
                },
                {
                    "role": "user",
                    "content": f"Here is the previous summary: {summary_chain[-1]}",
                },
                {
                    "role": "user",
                    "content": f"Missing Entities: {missing_entities}",
                },
            ],
        )
        output_summary = output.choices[0].message.content
        prev_summary = output_summary
        summary_chain.append(output_summary)

    return summary_chain

In [62]:
summarize_cod(llm_chain.llm.client, content, summary_steps=3, temperature=0.3)

Here are the missing entities:

* Tsunade (Naruto's friend and Jiraiya's student)
* Pa (one of the Six Paths of Pain, mentioned as a singer in the radio broadcast)

Let me know if you'd like me to proceed with generating a denser summary! 'NoneType' object is not iterable


["Here is a summary of the events and characters mentioned in this subtitle:\n\n**Characters:**\n\n* Naruto Uzumaki (main character)\n* Jiraiya (mentor of Naruto, appears as a radio DJ)\n* Nagato (one of the Six Paths of Pain, has Rinnegan eye)\n* Other members of the Six Paths of Pain (three unnamed clones with Rinnegan eyes)\n\n**Events:**\n\n* The Six Paths of Pain, led by Nagato, attack Jiraiya and Naruto\n* Jiraiya uses a genjutsu to bind the Six Paths' movements, but they are able to absorb his jutsu\n* Naruto and Jiraiya retreat and plan their next move\n* They discover that each of the Six Paths has only one type of ability, and that Nagato is using a jutsu to revive dead members\n* Jiraiya uses a genjutsu to paralyze the Six Paths, but Nagato reveals that he can still move his main body\n* Naruto and Jiraiya are trapped in a genjutsu paralysis, unable to move\n\n**Reason for Events:**\n\n* The Six Paths of Pain are seeking to destroy the ninja world, and Nagato believes that t

## Summarize all episodes

In [102]:
from tqdm import tqdm
from glob import glob

pattern = r"\[HorribleSubs\] Naruto Shippuuden - (\d{3}) \[\d{3}p\].ass"

folder_path = "/Users/nhuantran/Downloads/Naruto Shippuuden English Subs/"
output_name = "../data/naruto_shippuuden_subtitle.json"
with open(output_name, "w") as f:
    for idx, fpath in tqdm(enumerate(glob(folder_path+"/*.ass"))):
        # print("Process ", fpath)
        dialogues = parse_ass_file(fpath)
        df_dialogue = pd.DataFrame(dialogues, columns=["Layer", "Start", "End", "Style", "Name", "MarginL", "MarginR", "MarginV", "Effect","Text"])
        content = "\n".join(df_dialogue["Text"].tolist())

        file_name = fpath.split("/")[-1]
        match = re.search(pattern, file_name)
        episode_number = match.group(1)
        f.write(json.dumps({"episode_number": episode_number, "content": content}) + "\n")
        # output = llm_chain.invoke(content)
        # f.write(json.dumps({"episode_number": episode_number, "content": output["text"]}) + "\n")
        # break

        # if idx > 0 and idx % 10 == 0:
        #     f.flush()

426it [00:00, 1439.27it/s]


In [ ]:
from tqdm import tqdm
from glob import glob

pattern = r"\[HorribleSubs\] Naruto Shippuuden - (\d{3}) \[\d{3}p\].ass"

folder_path = "/Users/nhuantran/Downloads/Naruto Shippuuden English Subs/"
output_name = "../data/naruto_shippuuden_subtitle_summarize_by_episode.json"
with open(output_name, "w") as f:
    for idx, fpath in tqdm(enumerate(glob(folder_path+"/*.ass"))):
        print("Process ", fpath)
        dialogues = parse_ass_file(fpath)
        df_dialogue = pd.DataFrame(dialogues, columns=["Layer", "Start", "End", "Style", "Name", "MarginL", "MarginR", "MarginV", "Effect","Text"])
        content = "\n".join(df_dialogue["Text"].tolist())

        file_name = fpath.split("/")[-1]
        match = re.search(pattern, file_name)
        episode_number = match.group(1)
        output = llm_chain.invoke(content)
        f.write(json.dumps({"episode_number": episode_number, "content": output["text"]}) + "\n")

        if idx > 0 and idx % 10 == 0:
            f.flush()

In [4]:
# Query some dialogue

In [3]:
episodes = {}
with open("../data/naruto_shippuuden_subtitle_summarize_by_episode.json") as f:
    for line in f:
        tmp = json.loads(line.strip())
        episodes[tmp["episode_number"]] = tmp["content"]

In [4]:
print(episodes["140"])

Characters:
- Madara Uchiha: A character from the past who reveals the truth about Itachi to Sasuke. He was a leader of the Uchiha clan and had a strong influence on the events that led to Itachi's actions.
- Itachi Uchiha: The older brother of Sasuke, who was accused of betraying the Hidden Leaf Village. He is known for annihilating his own clan and deserting the village.
- Sasuke Uchiha: A main character in Naruto Shippuden, seeking to avenge his family's death by killing Itachi.
- Hashirama Senju: The leader of the Senju clan and the first Hokage, who played a significant role in shaping the history of the Hidden Leaf Village.
- Third Hokage: A character who opposed the mistreatment of the Uchiha clan but was not able to change the council's decision.
- Danzo: A member of the Anbu Black Ops who played a significant role in the discrimination against the Uchiha clan.

Events:
Description: Madara Uchiha reveals the truth about Itachi and the history of the Uchiha clan to Sasuke.
Chara

In [18]:
import mistletoe

In [19]:
rendered = mistletoe.markdown(episodes["140"])

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from optimum.onnxruntime import ORTModelForFeatureExtraction
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", device="mps", trust_remote_code=True)

/opt/homebrew/Caskroom/miniconda/base/envs/semantic_search/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/Caskroom/miniconda/base/envs/semantic_search/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>


In [6]:
embeddings = model.encode([episodes[key] for key in episodes.keys()], convert_to_tensor=True, batch_size=32, show_progress_bar=True, normalize_embeddings=True)

Batches: 100%|██████████| 14/14 [00:26<00:00,  1.88s/it]


In [7]:
def search_in_dialogue(index, question, top_k=5):
    question_embedding = model.encode([question], convert_to_numpy=True, normalize_embeddings=True)
    scores, question_indices = index.search(question_embedding, top_k)
    return question_indices[0], scores[0]

In [8]:
import faiss
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings.detach().cpu().numpy())

In [14]:
indices, scores = search_in_dialogue(index, "Jiraiya")
for idx, score in zip(indices, scores):
    print(idx, score)
    # format 3 digits
    print(episodes[str(idx).zfill(3)])

299 0.6416271
Characters:
- Nagato (Pain): A powerful ninja with the ability to absorb ninjutsu and control corpses using his Rinnegan. He uses jutsu such as Jigokudo, Planetary Devastation, and Reanimation Jutsu.
- Itachi Uchiha: A strong and skilled ninja known for his Totsuka Blade, which can seal away anything it pierces into an eternal genjutsu. He wields powerful Shuriken jutsu and Susano'o.
- Naruto Uzumaki: The protagonist with the Nine Tails Chakra Mode, Shadow Clone Jutsu, and strong determination.
- Killer Bee: A powerful summoner and friend of Naruto who summons a giant clam.

Events:
- Nagato uses his absorption jutsu to attack Itachi, who responds with his Shuriken jutsu, hitting Nagato's Rinnegan.
- The characters engage in a battle, discussing their jutsu and strategies. Itachi reveals the weaknesses in Nagato's Reanimation Jutsu.
- Naruto arrives on the scene and uses his Shadow Clone Jutsu to fight against the reanimated ninjas while urging others to stop the Reanimat

# Convert into sub-prompt

In [29]:
def get_chain(
    llm_url, model_name, template, temperature: float = 0.5, max_tokens: int = -1
):
    client = ChatOpenAI(
        temperature=temperature,
        base_url=llm_url,
        model_name=model_name,
        api_key=api_key,
        max_tokens=max_tokens,
    )
    map_prompt = PromptTemplate.from_template(template)
    return LLMChain(llm=client, prompt=map_prompt)


class ChainsList:
    def __init__(self, chains, document_variable: str = "content") -> None:
        self.chains = chains
        self.document_variable = document_variable

    def invoke(self, input, return_intermediate_steps=False):
        outputs = []
        for chain in self.chains:
            output = chain.invoke(input)
            outputs.append(output)
            input[self.document_variable] = output["text"]

        return outputs[-1] if not return_intermediate_steps else outputs

In [22]:
event_template = """You are manga nerd and you are watching Naruto
# Instruction
Convert the dialog to narrative and formal style for each event. Return in list of dialogues with number

# Guideline
- Both contents are in English
- Be informative, concise

### Input
{content}

### Response
"""

plot_template = """You are manga nerd and you are watching Naruto
# Instruction
Convert the dialogue into a plot

# Guideline
- Both contents are in English
- Be informative, concise

### Input
{content}

### Response
"""

character_template = """You are manga nerd and you are watching Naruto
### Instruction
Extract characters from the dialogue. Return Python list only. No furthur explained

### Guideline
- Both contents are in English
- Be informative, concise

### Input
{content}

### Response
"""

events_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
### Instruction
#### Summarize events in this subtitle
In clear and concise language, summarize chunk of sentences will take about the interactive events among the characters.

#### Generate 
Generate summary events, characters info, why it happened, where it happened and how it happened

### Guideline
- Both contents are in English
- Be informative, concise
- Return in list with number.
    1. 
    2. 

### Input
{content}

### Response
"""

rewrite_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
### Instruction
Re-write this summary into coherence paragraphs.

### Guideline
- Both contents are in English
- Be informative, concise

### Input
{content}

### Response
"""

incremental_events_template = """You are manga nerd and you are watching Naruto. You are asked to summarize the events in the subtitle.
### Instruction
#### Summarize events in this subtitle
In clear and concise language, summarize chunk of sentences will take about the interactive events among the characters.


### Steps by steps
1. Analyze plots, events from previous episode if provided 
2. Identify plots, events, characters from the subtitle
3. Then, check again those are factually consistent with the reviews.
3. Generate summary events, characters info, why it happened, where it happened and how it happened

### Guideline
- Return only, no furthur explain or self-explain
- Both contents are in English
- Be informative, concise
- Return in list with number. Example: 
    1. Event is 
    2. Naruto trains

### Previous Episode summary
{prev}
    
### Input
{content}

### Response
"""

In [30]:
model_name = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q8_0.gguf"
plot_llm_chain = get_chain(llm_url, model_name, incremental_events_template)
rewrite_llm_chain = get_chain(llm_url, model_name, rewrite_template)
chain_list = ChainsList([plot_llm_chain, rewrite_llm_chain])

In [38]:
from glob import glob

prev, current = "", ""

with open("../data/naruto_subtitle_summarize_by_episode.json", "w") as f:
    for i, fpath in enumerate(
        sorted(glob("/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/*.srt"))
    ):
        print(fpath)
        episode_number = str(i + 1).zfill(3)
        subtitles = extract_subtitles(fpath)
        content = "\n".join(subtitles)
        output = chain_list.invoke({"content": content, "prev": prev})
        prev = output["text"]
        f.write(
            json.dumps({"episode_number": episode_number, "content": output["text"]})
            + "\n"
        )

/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E001 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E002 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E003 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E004 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E005 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E006 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E007 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E008 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E009 (R2DVD 960x728 x264 AC3).en.srt
/Users/nhu

In [45]:
season_one_path = "../data/naruto_subtitle_summarize_by_episode.json"
lines = [json.loads(l) for l in open(season_one_path).readlines()]

In [74]:
last_content = "-".join([lines[i]["content"] for i in range(-6, 0, 1)])

In [75]:
from tqdm import tqdm
from glob import glob

pattern = r"\[HorribleSubs\] Naruto Shippuuden - (\d{3}) \[\d{3}p\].ass"
prev = last_content
folder_path = "/Users/nhuantran/Downloads/Naruto Shippuuden English Subs/"
output_name = "../data/naruto_shippuuden_subtitle.json"
with open(output_name, "w") as f:
    for idx, fpath in tqdm(enumerate(sorted(glob(folder_path+"/*.ass")))):
        print("Process ", fpath)
        dialogues = parse_ass_file(fpath)
        df_dialogue = pd.DataFrame(dialogues, columns=["Layer", "Start", "End", "Style", "Name", "MarginL", "MarginR", "MarginV", "Effect","Text"])
        content = "-".join(df_dialogue["Text"].tolist())

        file_name = fpath.split("/")[-1]
        match = re.search(pattern, file_name)
        episode_number = match.group(1)
        output = chain_list.invoke({"content": content, "prev": prev})
        prev = output["text"]
        f.write(
            json.dumps({"episode_number": episode_number, "content": output["text"]})
            + "\n"
        )

        if idx > 0 and idx % 10 == 0:
            f.flush()

0it [00:00, ?it/s]

Process  /Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - 001 [480p].ass


1it [01:03, 63.38s/it]

Process  /Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - 002 [480p].ass


2it [01:52, 54.99s/it]

...

424it [5:44:24, 46.54s/it]

Process  /Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - 425 [720p].ass


425it [5:45:22, 50.02s/it]

Process  /Users/nhuantran/Downloads/Naruto Shippuuden English Subs/[HorribleSubs] Naruto Shippuuden - 426 [720p].ass


426it [5:45:54, 48.72s/it]


In [81]:
from elasticsearch import Elasticsearch
from tqdm import tqdm
es_client = Elasticsearch()

In [82]:
season_one_path = "../data/naruto_subtitle_summarize_by_episode.json"
season_one_dialogue_format_path = "/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E{episode} (R2DVD 960x728 x264 AC3).en.srt"
lines = [json.loads(l) for l in open(season_one_path).readlines()]

for line in tqdm(lines):
    episode_number = line["episode_number"]
    summarize_episode_content  = line["content"]
    fpath = season_one_dialogue_format_path.format(episode=episode_number)
    subtitles = extract_subtitles(fpath)
    content = "-".join(subtitles)
    es_client.index(index="naruto_episode", body={"content": summarize_episode_content, "dialogue": content, "type": "small", "episode_number": episode_number})

  0%|          | 0/135 [00:00<?, ?it/s]/var/folders/_q/zxnjn9rd7vj0h6477qfvbfw00000gn/T/ipykernel_64085/4082663910.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.index(index="naruto_episode", body={"content": summarize_episode_content, "dialogue": content, "type": "small", "episode_number": episode_number})
100%|██████████| 135/135 [00:00<00:00, 142.04it/s]


In [83]:
season_two_path = "../data/naruto_shippuuden_subtitle.json"
season_one_dialogue_format_path = "/Users/nhuantran/Downloads/Naruto English Subtitles 1-135/(SB-RAW)NARUTO E{episode} (R2DVD 960x728 x264 AC3).en.srt"
lines = [json.loads(l) for l in open(season_two_path).readlines()]

for line in tqdm(lines):
    episode_number = int(line["episode_number"])
    summarize_episode_content = line["content"]
    es_client.index(index="naruto_episode", body={"content": summarize_episode_content, "type": "shippuden", "episode_number": episode_number})

  0%|          | 0/426 [00:00<?, ?it/s]/var/folders/_q/zxnjn9rd7vj0h6477qfvbfw00000gn/T/ipykernel_64085/3202105387.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.index(index="naruto_episode", body={"content": summarize_episode_content, "type": "shippuden", "episode_number": episode_number})
100%|██████████| 426/426 [00:00<00:00, 617.66it/s]
